In [1]:
import networkx as nx 
from glob import glob
import pandas as pd 
import pickle as pkl
import csv
from tqdm import tqdm
import bz2
import json
import requests as rq
import time
import lzma

In [2]:
import logging
logger = logging.getLogger()
fhandler = logging.FileHandler(filename='Geo_Parsing.log', mode='a')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.setLevel(logging.DEBUG)

In [3]:
submissionFiles = '/10TBdrive/datashare/geo_located_Reddit_crawl/yearlySubmissions/'
years = ['2005','2006','2007','2008','2009','2010','2011','2012','2013','2017']

selected_submissions = {}
for year in tqdm(years):
    yearlist = pkl.load(open(submissionFiles + year + ".pkl", 'rb'))
    for k in yearlist:
        subreddit = yearlist[k].split('/')[2]
        selected_submissions[k] = subreddit

100%|██████████| 10/10 [02:10<00:00, 13.02s/it]


In [6]:
selected_submissions[list(selected_submissions.keys())[10000125]]

'depaul'

In [7]:
graphs_2017 = glob("/10TBdrive/datashare/Reddit/graphs/*.pkl")
geo_graphs = glob("/10TBdrive/datashare/geo_located_Reddit_crawl/graphs/*.pkl")

total_graphs = graphs_2017 + geo_graphs

In [8]:
len(total_graphs)

21353000

In [14]:
total_graphs[0]

'/10TBdrive/datashare/Reddit/graphs/6h89c5.pkl'

In [16]:
graph_path_map= {}
for k in tqdm(total_graphs):
    graph_id = k.split('/')[-1].split('.')[0]
    graph_path_map[graph_id] = k
    

100%|██████████| 21353000/21353000 [00:33<00:00, 645153.63it/s]


In [18]:
common_graphs = selected_submissions.keys() & graph_path_map.keys()

In [19]:
len(common_graphs)

16470632

In [27]:
common_graphs[1]

'6faxdy'

In [ ]:
g = pkl.load(open(graph_path_map[common_graphs[0]],'rb'))

In [28]:
g.nodes(data=True)

[('t3_13717v',
  {'affects': 3,
   'author': 'Allurex',
   'depth': -1,
   'downs': 0,
   'text': '',
   'time': 1352920421.0,
   'ups': 83}),
 ('t1_c71cq0z',
  {'affects': 3,
   'author': 'Allurex',
   'depth': 0,
   'downs': 0,
   'text': 'Re-discovered my favorite internet video of all time. Original upload was probably 2005 or maybe even before that.',
   'time': 1352920866.0,
   'ups': 5}),
 ('t1_c71ocm4',
  {'affects': 3,
   'author': 'MrSupo',
   'depth': 1,
   'downs': 0,
   'text': 'It was around 2004. My older brother was in highschool and it was popular there, he interupted my Runescape session to show me.',
   'time': 1352964952.0,
   'ups': 1}),
 ('t1_c71f4we',
  {'affects': 3,
   'author': 'dtrmp4',
   'depth': 0,
   'downs': 0,
   'text': 'knew exactly what is was from the title',
   'time': 1352928717.0,
   'ups': 8}),
 ('t1_c71o9ms',
  {'affects': 3,
   'author': 'ShadedNature',
   'depth': 0,
   'downs': 0,
   'text': "Thank you so much.  This was me and my friend's f

In [ ]:
# g.edges()

In [ ]:
with open('/10TBdrive/datashare/Reddit/AllParsedData_2005_to_2012_and_2017.csv', 'w') as f:  # Just use 'w' mode in 3.x
    w = csv.DictWriter(f, ['author','downs','ups','text','time','src_id','dest_id','dest_author','depth','subreddit'],delimiter='|')
    w.writeheader()
    count = 0
    total = len(common_graphs)
    for gra in tqdm(common_graphs):
        g = pkl.load(open(graph_path_map[gra],'rb'))
        gid = gra
        logging.debug(f"Parsing {count} of {total} graphs")
        try:
            subreddit = selected_submissions[gid]
            nodes = g.nodes(data=True)
            node_dict = {n[0] : n[1]  for n in nodes}
            for u , v in g.edges():
                if node_dict[u]['author']!='[deleted]' and node_dict[v]['author']!='[deleted]' :
                    row= {'author': node_dict[u]['author'] , 'downs' : node_dict[u]['downs'] , 'ups': node_dict[u]['ups'] , 'text':node_dict[u]['text'] , 'time':node_dict[u]['time'] , 'src_id' : u , 'dest_id' : v , 'dest_author' : node_dict[v]['author'] ,'depth':node_dict[u]['depth'],'subreddit':subreddit }
                    w.writerow(row)
                else:
                    continue
        except:
            print("Some graph is malformed!! ")
            logging.debug("Some graph is malformed!!")
        count+=1

  0%|          | 1349/16470632 [03:09<782:59:32,  5.84it/s] 

Some graph is malformed!! 


  0%|          | 4608/16470632 [10:34<789:41:46,  5.79it/s] 

In [ ]:
# reddit_data = pd.read_csv('/10TBdrive/datashare/Reddit/AllParsedData_2017_V3.csv',sep='|')

In [ ]:
# len(reddit_data)

In [ ]:
# reddit_data.head()

In [ ]:
# len(reddit_data['author'].unique())